# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import utils
import pathing

import osmnx as ox
import networkx
import random
import time
from tqdm import tqdm


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


# Methods

In [5]:
def get_node(od: pathing.OriginDestination, x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]
    
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)


In [6]:
def evaluate_travel_time_accuracy(benchmark_times, calculated_times):
    total_diff = 0
    total_diff_percentage = 0

    for destination, benchmark in benchmark_times.items():
        if destination in calculated_times:
            calculated = calculated_times[destination]
            diff = abs(benchmark - calculated)
            diff_percentage = (diff / benchmark) * 100

            total_diff += diff
            total_diff_percentage += diff_percentage

            print(f"Destination {destination}:")
            print(f"  Benchmark Time: {benchmark} minutes")
            print(f"  Calculated Time: {calculated} minutes")
            print(f"  Difference: {diff} minutes ({diff_percentage:.2f}%)")
            print()
        else:
            print(f"Destination {destination} not found in calculated times.")

    avg_diff_percentage = total_diff_percentage / len(benchmark_times)
    print(f"Total Discrepancy: {total_diff} minutes across all destinations.")
    print(f"Average Discrepancy Percentage: {avg_diff_percentage:.2f}%")


In [7]:
def calculate_error(od: pathing.OriginDestination, benchmark_times, start_node):
    calculated_times = {}
    
    # Assuming utils and get_node functions are defined and work as intended
    for (lon, lat), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        x, y = utils.geographic_to_utm(lon, lat)
        destination_node = get_node(od, x, y)

        # Calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            destination_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # Store the calculated travel time
        calculated_times[(lon, lat)] = total_travel_time

    # Calculate MSE
    errors = [abs(benchmark_times[key] - calculated_times[key]) for key in benchmark_times]

    mse = sum(error ** 2 for error in errors) / len(errors)
    
    return mse


In [8]:
def gen_linkers(road_type_factors, link_factor):
    linkables = [
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
    ]

    for linkable in linkables:
        road_type_factors[linkable + "_link"] = road_type_factors[linkable] * link_factor
    
    return road_type_factors


In [9]:
def optimize_parameters(od: pathing.OriginDestination, benchmark_times, start_node, max_duration_minutes):
    start_time = time.time()
    max_duration_seconds = max_duration_minutes * 60
    
    # Define parameter bounds
    param_bounds = {
        "default_intersection_penalty": (5, 20),
        "traffic_signal_penalty": (5, 30),
        "linkable_factor": (0.75, 1.0),
        "road_type_factors": {
            "motorway": (0.75, 1.0),
            "trunk": (0.75, 1.0),
            "primary": (0.75, 1.0),
            "secondary": (0.5, 0.75),
            "tertiary": (0.5, 0.75),
            "unclassified": (0.5, 0.75),
            "residential": (0.25, 0.75),
            "living_street": (0.25, 0.75)
        }
    }
    
    # Initialize the best solution
    best_solution = None
    best_error = float("inf")
    
    # Progress bar
    with tqdm(total=max_duration_seconds, desc="Optimizing Parameters") as pbar:
        while time.time() - start_time < max_duration_seconds:
            # Generate a random solution within the bounds
            current_solution = {
                "default_intersection_penalty": random.randint(*param_bounds["default_intersection_penalty"]),
                "traffic_signal_penalty": random.randint(*param_bounds["traffic_signal_penalty"]),
                "road_type_factors": {k: random.uniform(*v) for k, v in param_bounds["road_type_factors"].items()}
            }
            linkable_factor = random.uniform(*param_bounds["linkable_factor"])

            current_solution["road_type_factors"] = gen_linkers(current_solution["road_type_factors"], linkable_factor)
            
            # Set graph weights with the current solution
            od.set_graph_weights_v2(**current_solution)
            
            # Calculate the error for the current solution
            current_error = calculate_error(od, benchmark_times, start_node)
            
            # Update best solution if the current one is better
            if current_error < best_error:
                best_solution = current_solution
                best_error = current_error
                tqdm.write(
                    f"    New best solution:\n"
                    f"        Default Intersection Penalty: {best_solution['default_intersection_penalty']}\n"
                    f"        Traffic Signal Penalty: {best_solution['traffic_signal_penalty']}\n"
                    f"        Road Type Factors:\n"
                    + "\n".join([f"            {k}: {v}" for k, v in best_solution['road_type_factors'].items()]) +
                    f"\n        With error: {best_error}"
                )
            
            # Update progress bar
            pbar.update(time.time() - start_time - pbar.n)
            
            # Exit loop if the maximum duration is exceeded
            if pbar.n >= max_duration_seconds:
                break
    
    return best_solution, best_error


# Main

In [10]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

od.get_graph()

central_depot_grid_id = 22620006649000
central_depot_x, central_depot_y = utils.id_to_utm(central_depot_grid_id)
od.node_validator = ox.distance.nearest_nodes(od.graph, central_depot_x, central_depot_y)


In [11]:
x, y = utils.geographic_to_utm(10.7343661, 59.9369806)
start_node = get_node(od, x, y)

benchmark_times = {
    (10.6572104, 59.6800045): 40,
    (10.6661424, 59.8598299): 60,
    (10.4852080, 59.8073283): 35,
    (11.8086592, 60.2271721): 68,
    (10.7193651, 60.1973847): 53,
    (10.7622022, 60.0035569): 14
}


In [12]:
best_solution, best_error = optimize_parameters(
    od,
    benchmark_times,
    start_node,
    max_duration_minutes=1
)


Optimizing Parameters:  15%|█▍        | 8.723611831665039/60 [00:08<00:51,  1.00it/s]

    New best solution:
        Default Intersection Penalty: 14
        Traffic Signal Penalty: 16
        Road Type Factors:
            motorway: 0.7523587491819068
            trunk: 0.7862210745507554
            primary: 0.7705193704887314
            secondary: 0.5140308158211166
            tertiary: 0.6320448341025829
            unclassified: 0.7155955425163079
            residential: 0.7194432660756624
            living_street: 0.4067165954020093
            motorway_link: 0.7059187111810864
            trunk_link: 0.7376908532714984
            primary_link: 0.7229583513807936
            secondary_link: 0.48230178941412305
            tertiary_link: 0.5930312835246666
        With error: 178.18604920415916


Optimizing Parameters:  21%|██        | 12.357666969299316/60 [00:12<00:47,  1.00it/s]

    New best solution:
        Default Intersection Penalty: 11
        Traffic Signal Penalty: 10
        Road Type Factors:
            motorway: 0.8532718504756684
            trunk: 0.8448146692720017
            primary: 0.9236827478601958
            secondary: 0.6878716543520392
            tertiary: 0.5141160986750725
            unclassified: 0.6428882122654791
            residential: 0.37815935925124433
            living_street: 0.7281979154856979
            motorway_link: 0.7347357272236807
            trunk_link: 0.7274534136462733
            primary_link: 0.795365175933905
            secondary_link: 0.5923128483790452
            tertiary_link: 0.4426953325916644
        With error: 75.44653564614828


Optimizing Parameters:  27%|██▋       | 16.20295739173889/60 [00:16<00:43,  1.00it/s] 

...

Optimizing Parameters:  83%|████████▎ | 49.53409957885742/60 [00:49<00:10,  1.00it/s] 

    New best solution:
        Default Intersection Penalty: 10
        Traffic Signal Penalty: 18
        Road Type Factors:
            motorway: 0.8266669543563125
            trunk: 0.9895018372104942
            primary: 0.8094856939227552
            secondary: 0.5341171531025775
            tertiary: 0.7400324282726038
            unclassified: 0.5628282547805638
            residential: 0.5669180967251916
            living_street: 0.5460711797725011
            motorway_link: 0.7210574902781329
            trunk_link: 0.8630896730596429
            primary_link: 0.7060711932418828
            secondary_link: 0.46588190310634603
            tertiary_link: 0.6454908142181278
        With error: 36.101032570702024


Optimizing Parameters: 100%|█████████▉| 59.926435470581055/60 [00:59<00:00,  1.00it/s]

    New best solution:
        Default Intersection Penalty: 9
        Traffic Signal Penalty: 6
        Road Type Factors:
            motorway: 0.9614682201862459
            trunk: 0.9299403221139196
            primary: 0.8931321884184713
            secondary: 0.5232343633373289
            tertiary: 0.7194797357417648
            unclassified: 0.5785951162356147
            residential: 0.3568164275409318
            living_street: 0.30230075317088123
            motorway_link: 0.9324388163904462
            trunk_link: 0.9018628333838068
            primary_link: 0.8661660397759258
            secondary_link: 0.5074364604069386
            tertiary_link: 0.6977566383650288
        With error: 32.138742909857626


Optimizing Parameters: 63.89891958236694it [01:03,  1.00it/s]                         


In [14]:
od.set_graph_weights_v2(**best_solution)

calculated_times = {}

for lon, lat in benchmark_times.keys():
    # gets destination node for UTM coordinates
    x, y = utils.geographic_to_utm(lon, lat)
    destination_node = get_node(od, x, y)

    # get path of nodes from start node to destination node
    shortest_time_path = ox.shortest_path(
        od.graph,
        start_node,
        destination_node,
        weight='time'
    )

    # caclulate travel time between start and destination node and cache the result
    total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))
    calculated_times[(lon, lat)] = total_travel_time
    
evaluate_travel_time_accuracy(benchmark_times, calculated_times)


Destination (10.6572104, 59.6800045):
  Benchmark Time: 40 minutes
  Calculated Time: 44.53320427050637 minutes
  Difference: 4.533204270506367 minutes (11.33%)

Destination (10.6661424, 59.8598299):
  Benchmark Time: 60 minutes
  Calculated Time: 58.32057058848948 minutes
  Difference: 1.6794294115105188 minutes (2.80%)

Destination (10.485208, 59.8073283):
  Benchmark Time: 35 minutes
  Calculated Time: 32.67541773845802 minutes
  Difference: 2.324582261541977 minutes (6.64%)

Destination (11.8086592, 60.2271721):
  Benchmark Time: 68 minutes
  Calculated Time: 79.0054808214622 minutes
  Difference: 11.0054808214622 minutes (16.18%)

Destination (10.7193651, 60.1973847):
  Benchmark Time: 53 minutes
  Calculated Time: 55.003349126789395 minutes
  Difference: 2.003349126789395 minutes (3.78%)

Destination (10.7622022, 60.0035569):
  Benchmark Time: 14 minutes
  Calculated Time: 20.238936994930008 minutes
  Difference: 6.2389369949300075 minutes (44.56%)

Total Discrepancy: 27.78498288